In [31]:
import pathlib
import pickle
import pandas as pd
import numpy as np
import scipy
import sklearn
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import mean_squared_error
import mlflow
import xgboost as xgb
from prefect import flow, task
from prefect.artifacts import create_markdown_artifact
from datetime import date

In [32]:
@task(retries=3, retry_delay_seconds=2,name = "Read taxi data 2023 files")
def read_data(filename: str) -> pd.DataFrame:
    """Read data into DataFrame"""
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df["duration"] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ["PULocationID", "DOLocationID"]
    df[categorical] = df[categorical].astype(str)

    return df

In [33]:
@task(name = "Create new features from data")
def add_features(
    df_train: pd.DataFrame, df_val: pd.DataFrame
) -> tuple(
    [
        scipy.sparse._csr.csr_matrix,
        scipy.sparse._csr.csr_matrix,
        np.ndarray,
        np.ndarray,
        sklearn.feature_extraction.DictVectorizer,
    ]
):
    """Add features to the model"""
    df_train["PU_DO"] = df_train["PULocationID"] + "_" + df_train["DOLocationID"]
    df_val["PU_DO"] = df_val["PULocationID"] + "_" + df_val["DOLocationID"]

    categorical = ["PU_DO"]  #'PULocationID', 'DOLocationID']
    numerical = ["trip_distance"]

    dv = DictVectorizer()

    train_dicts = df_train[categorical + numerical].to_dict(orient="records")
    X_train = dv.fit_transform(train_dicts)

    val_dicts = df_val[categorical + numerical].to_dict(orient="records")
    X_val = dv.transform(val_dicts)

    y_train = df_train["duration"].values
    y_val = df_val["duration"].values
    return X_train, X_val, y_train, y_val, dv

In [34]:
@task(log_prints=True,name = "Train best model and test it")
def train_best_model(
    X_train: scipy.sparse._csr.csr_matrix,
    X_val: scipy.sparse._csr.csr_matrix,
    y_train: np.ndarray,
    y_val: np.ndarray,
    dv: sklearn.feature_extraction.DictVectorizer,
) -> None:
    """train a model with best hyperparams and write everything out"""

    with mlflow.start_run():
        train = xgb.DMatrix(X_train, label=y_train)
        valid = xgb.DMatrix(X_val, label=y_val)

        best_params = {
            "learning_rate": 0.09585355369315604,
            "max_depth": 30,
            "min_child_weight": 1.060597050922164,
            "objective": "reg:linear",
            "reg_alpha": 0.018060244040060163,
            "reg_lambda": 0.011658731377413597,
            "seed": 42,
        }

        mlflow.log_params(best_params)

        booster = xgb.train(
            params=best_params,
            dtrain=train,
            num_boost_round=100,
            evals=[(valid, "validation")],
            early_stopping_rounds=20,
        )

        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

        pathlib.Path("models").mkdir(exist_ok=True)
        with open("models/preprocessor.b", "wb") as f_out:
            pickle.dump(dv, f_out)
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")
        
        markdown__rmse_report = f"""# RMSE Report

        ## Summary

        Duration Prediction 

        ## RMSE XGBoost Model

        | Region    | RMSE |
        |:----------|-------:|
        | {date.today()} | {rmse:.2f} |
        """

        create_markdown_artifact(
            key="duration-model-report", markdown=markdown__rmse_report
        )
        
    return None

In [35]:
@flow(name = "Overall Workflow for pipeline")
def main_flow(
    train_path: str = "C:/Project/MLOPs/Data/green_tripdata_2023-01.parquet",
    val_path: str = "C:/Project/MLOPs/Data/green_tripdata_2023-02.parquet",
) -> None:
    """The main training pipeline"""

    # MLflow settings
    mlflow.set_tracking_uri("sqlite:///mlflow.db")
    mlflow.set_experiment("nyc-taxi-experiment")

    # Load
    df_train = read_data(train_path)
    df_val = read_data(val_path)

    # Transform
    X_train, X_val, y_train, y_val, dv = add_features(df_train, df_val)

    # Train
    train_best_model(X_train, X_val, y_train, y_val, dv)

In [36]:
if __name__ == "__main__":
    main_flow()

23:21:18.729 | INFO    | prefect.engine - Created flow run 'great-taipan' for flow 'Overall Workflow for pipeline'

23:21:19.003 | INFO    | Flow run 'great-taipan' - Created task run 'Read taxi data 2023 files-0' for task 'Read taxi data 2023 files'

23:21:19.012 | INFO    | Flow run 'great-taipan' - Executing 'Read taxi data 2023 files-0' immediately...

23:21:19.548 | INFO    | Task run 'Read taxi data 2023 files-0' - Finished in state Completed()

23:21:19.589 | INFO    | Flow run 'great-taipan' - Created task run 'Read taxi data 2023 files-1' for task 'Read taxi data 2023 files'

...

23:21:25.656 | INFO    | Task run 'Train best model and test it-0' - [67]       validation-rmse:5.67631

23:21:25.698 | INFO    | Task run 'Train best model and test it-0' - [68]       validation-rmse:5.67631

23:21:25.794 | INFO    | Task run 'Train best model and test it-0' - [69]       validation-rmse:5.67631

23:21:32.489 | INFO    | Task run 'Train best model and test it-0' - Finished in state Completed()

23:21:32.547 | INFO    | Flow run 'great-taipan' - Finished in state Completed('All states completed.')

In [37]:
from prefect_email import EmailServerCredentials
email_credentials_block = EmailServerCredentials.load("emailservercredentials")

In [38]:
email_credentials_block

<coroutine object Block.load at 0x000001F845818040>